<a href="https://colab.research.google.com/github/eisbetterthanpi/python/blob/master/draw_desmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://github.com/vigneshsaravanakumar404/ImageToEquations
!pip install svgpathtools
!pip install svgpathtools
!pip install numpy
# !pip install svgutils


In [2]:
# @title Functions
import re
import webbrowser
import numpy as np
import svgpathtools
from svgpathtools import Line, CubicBezier

# Detect the types of segments
def _tokenize_path(pathfinder):
    FLOAT_RE = re.compile("[-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?")
    for x in re.compile("([MmZzLlHhVvCcSsQqTtAa])").split(pathfinder):
        if x in set("MmZzLlHhVvCcSsQqTtAa"):
            yield x
        for token in FLOAT_RE.findall(x):
            yield token


# transform into a complex number in the for (a + bi)
def aplusbiFormat(real, imaginary):
    return real + imaginary * 1j


# Convert points to equations from the bezier points
def extract_path(pathfinder, current_pos=0j):
    # Variables
    elements = list(_tokenize_path(pathfinder))
    elements.reverse()
    segments = []
    start_pos = None
    command = None
    # Loop through all the paths
    while elements:
        if elements[-1] in set("MmZzLlHhVvCcSsQqTtAa"):
            command = elements.pop()
            absolute = command in set("MZLHVCSQTA")
            command = command.upper()
        else:
            if command is None: raise ValueError("idk what happened so im just gonna say error. Error!")
        if command == "M":
            x = elements.pop()
            y = elements.pop()
            pos = float(x) + float(y) * 1j
            if absolute: current_pos = pos
            else: current_pos += pos
            start_pos = current_pos
            command = "L"
        elif command == "Z":
            if not (current_pos == start_pos): segments.append(Line(current_pos, start_pos))
            current_pos = start_pos
            command = None
        elif command == "L":
            x = elements.pop()
            y = elements.pop()
            pos = float(x) + float(y) * 1j
            if not absolute: pos += current_pos
            segments.append(Line(current_pos, pos))
            current_pos = pos
        elif command == "C":
            control1 = float(elements.pop()) + float(elements.pop()) * 1j
            control2 = float(elements.pop()) + float(elements.pop()) * 1j
            final = float(elements.pop()) + float(elements.pop()) * 1j
            if not absolute:
                control1 += current_pos
                control2 += current_pos
                final += current_pos
            segments.append(CubicBezier(current_pos, control1, control2, final))
            current_pos = final
    return segments


def run(data,off_x=0,off_y=0,scale=1):
    # get all the text in between the <path and ></path>
    pathArray = re.findall(r'<path d="(.*?)"', data, re.DOTALL)
    pathString = ""
    for path in pathArray:
        pathString += path
    path = extract_path(pathString)  # Get the path from the SVG file
    equations, regularEquations = [], []
    for segment in path:
        # Iterate through each segment, a set of 4 points, in the SVG file and check what type of segment it is
        if isinstance(segment, svgpathtools.path.Line):
            # Extract the start and end points from the line segment
            start = aplusbiFormat(segment.start.real, segment.start.imag)
            end = aplusbiFormat(segment.end.real, segment.end.imag)
            # check to make sure line doesn't have undefined slope to prevent mathematical errors
            if end.real - start.real != 0 and end.imag - start.imag != 0:
                # calculate the slope and y-intercept of the line segment
                m = (end.imag - start.imag) / (end.real - start.real)
                b = start.imag - m * start.real
                # calculate the bounds of the line segment in the x direction
                xMin = min(start.real, end.real)
                xMax = max(start.real, end.real)
                # calculate the bounds of the line segment in the y direction
                yMin = min(start.imag, end.imag)
                yMax = max(start.imag, end.imag)
                # Convert the linear equation into the form y=mx+b and put it in latex format
                equations.append("y="+ str(m)+ "x+"+ str(b)+ "\\\\left\\\\{"+ str(xMin)+ "\\\\le x \\\\le "+ str(yMin)+ "\\\\right\\\\}\\\\left\\\\{"+ str(yMin)+ "\\\\le y \\\\le "+ str(yMax)+ "\\\\right\\\\}")
                # Convert the linear equation into the form y=mx+b and put it in lambda format
                regularEquations.append(lambda x: m * x + b)
            if end.real - start.real == 0:
                # calculate the bounds of the line segment in the x direction
                xMin = min(start.real, end.real)
                xMax = max(start.real, end.real)
                # calculate the bounds of the line segment in the y direction
                yMin = min(start.imag, end.imag)
                yMax = max(start.imag, end.imag)
                # Convert the linear equation into the form x=c and put it in latex format
                equations.append("x="+ str(start.real)+ "\\\\left\\\\{"+ str(xMin)+ "\\\\le x \\\\le "+ str(yMin)+ "\\\\right\\\\}\\\\left\\\\{"+ str(yMin)+ "\\\\le y \\\\le "+ str(yMax)+ "\\\\right\\\\}")
                # Convert the linear equation into the form x=c and put it in lambda format
                regularEquations.append(lambda x: start.real)
            else:
                yMin = min(start.imag, end.imag)
                yMax = max(start.imag, end.imag)
                # if the slope is undefined, then the line is vertical and the equation is in the form x=a
                equations.append("x="+ str(start.real)+ "\\\\left\\\\{"+ str(yMin)+ "\\\\le y \\\\le "+ str(yMax)+ "\\\\right\\\\}")
                # if the slope is undefined, then the line is vertical and the equation is in the form x=a
                regularEquations.append(lambda x: start.real)
        elif isinstance(segment, svgpathtools.path.CubicBezier):
            # extract the bezier points from the segment
            p0 = aplusbiFormat(segment.start.real, segment.start.imag)
            p1 = aplusbiFormat(segment.control1.real, segment.control1.imag)
            p2 = aplusbiFormat(segment.control2.real, segment.control2.imag)
            p3 = aplusbiFormat(segment.end.real, segment.end.imag)
            # Convert the bezier points into a parametric equation in latex format
            equations.append("\\\\left((1-t)^3*"+ str(p0.real)+ "+3*t*(1-t)^2*"+ str(p1.real)+ "+3*t^2*(1-t)*"+ str(p2.real)+ "+t^3*"+ str(p3.real)+ ", (1-t)^3*"+ str(p0.imag)+ "+3*t*(1-t)^2*"+ str(p1.imag)+ "+3*t^2*(1-t)*"+ str(p2.imag)+ "+t^3*"+ str(p3.imag)+ ")\\\\right)")
            # Convert the bezier points into a parametric equation in lambda format
            regularEquations.append(lambda t: (1 - t) ** 3 * p0+ 3 * t * (1 - t) ** 2 * p1+ 3 * t ** 2 * (1 - t) * p2+ t ** 3 * p3)

        elif isinstance(segment, svgpathtools.path.QuadraticBezier):
            # Quadratic Bezier segment
            p0 = aplusbiFormat(segment.start.real, segment.start.imag)
            p1 = aplusbiFormat(segment.control.real, segment.control.imag)
            p2 = aplusbiFormat(segment.end.real, segment.end.imag)
            # Convert the bezier points into a parametric equation in latex format
            equations.append("\\\\left((1-t)^2*"+ str(p0.real)+ "+2*t*(1-t)*"+ str(p1.real)+ "+t^2*"+ str(p2.real)+ ", (1-t)^2*"+ str(p0.imag)+ "+2*t*(1-t)*"+ str(p1.imag)+ "+t^2*"+ str(p2.imag)+ ")\\\\right))")
            # Convert the bezier points into a parametric equation in lambda format
            regularEquations.append(lambda t: (1 - t) ** 2 * p0 + 2 * t * (1 - t) * p1 + t ** 2 * p2)
        elif isinstance(segment, svgpathtools.path.Arc):
            # Elliptical arc segment
            p0 = aplusbiFormat(segment.start.real, segment.start.imag)
            p1 = aplusbiFormat(segment.end.real, segment.end.imag)
            r = aplusbiFormat(segment.radius.real, segment.radius.imag)
            # Convert the bezier points into a parametric equation in latex format
            equations.append("\\\\left("+ str(p0.real)+ "+"+ str(r.real)+ "*\\cos(t), "+ str(p0.imag)+ "+"+ str(r.imag)+ "*\\sin(t)\\\\right)")
            # Convert the bezier points into a parametric equation in lambda format
            regularEquations.append(lambda t: p0 + r * np.exp(1j * t))
        else: print("Unknown segment type: " + str(type(segment)))
    return equations


# def run(data,x_off=0,y_off=0,scale=1):
#     # get all the text in between the <path and ></path>
#     pathArray = re.findall(r'<path d="(.*?)"', data, re.DOTALL)
#     pathString = ""
#     for path in pathArray:
#         pathString += path
#     path = extract_path(pathString)  # Get the path from the SVG file
#     equations, regularEquations = [], []
#     for segment in path:
#         # Iterate through each segment, a set of 4 points, in the SVG file and check what type of segment it is
#         if isinstance(segment, svgpathtools.path.Line):
#             # Extract the start and end points from the line segment
#             start = aplusbiFormat(segment.start.real*scale+x_off, segment.start.imag*scale+y_off)
#             end = aplusbiFormat(segment.end.real*scale+x_off, segment.end.imag*scale+y_off)
#             # check to make sure line doesn't have undefined slope to prevent mathematical errors
#             if end.real - start.real != 0 and end.imag - start.imag != 0:
#                 # calculate the slope and y-intercept of the line segment
#                 m = (end.imag - start.imag) / (end.real - start.real)
#                 b = start.imag - m * start.real
#                 # calculate the bounds of the line segment in the x direction
#                 xMin = min(start.real, end.real)
#                 xMax = max(start.real, end.real)
#                 # calculate the bounds of the line segment in the y direction
#                 yMin = min(start.imag, end.imag)
#                 yMax = max(start.imag, end.imag)
#                 # Convert the linear equation into the form y=mx+b and put it in latex format
#                 equations.append("y="+ str(m)+ "x+"+ str(b)+ "\\\\left\\\\{"+ str(xMin)+ "\\\\le x \\\\le "+ str(yMin)+ "\\\\right\\\\}\\\\left\\\\{"+ str(yMin)+ "\\\\le y \\\\le "+ str(yMax)+ "\\\\right\\\\}")
#                 # Convert the linear equation into the form y=mx+b and put it in lambda format
#                 regularEquations.append(lambda x: m * x + b)
#             if end.real - start.real == 0:
#                 # calculate the bounds of the line segment in the x direction
#                 xMin = min(start.real, end.real)
#                 xMax = max(start.real, end.real)
#                 # calculate the bounds of the line segment in the y direction
#                 yMin = min(start.imag, end.imag)
#                 yMax = max(start.imag, end.imag)
#                 # Convert the linear equation into the form x=c and put it in latex format
#                 equations.append("x="+ str(start.real)+ "\\\\left\\\\{"+ str(xMin)+ "\\\\le x \\\\le "+ str(yMin)+ "\\\\right\\\\}\\\\left\\\\{"+ str(yMin)+ "\\\\le y \\\\le "+ str(yMax)+ "\\\\right\\\\}")
#                 # Convert the linear equation into the form x=c and put it in lambda format
#                 regularEquations.append(lambda x: start.real)
#             else:
#                 yMin = min(start.imag, end.imag)
#                 yMax = max(start.imag, end.imag)
#                 # if the slope is undefined, then the line is vertical and the equation is in the form x=a
#                 equations.append("x="+ str(start.real)+ "\\\\left\\\\{"+ str(yMin)+ "\\\\le y \\\\le "+ str(yMax)+ "\\\\right\\\\}")
#                 # if the slope is undefined, then the line is vertical and the equation is in the form x=a
#                 regularEquations.append(lambda x: start.real)
#         elif isinstance(segment, svgpathtools.path.CubicBezier):
#             # extract the bezier points from the segment
#             p0 = aplusbiFormat(segment.start.real*scale+x_off, segment.start.imag*scale+y_off)
#             p1 = aplusbiFormat(segment.control1.real*scale, segment.control1.imag*scale)
#             p2 = aplusbiFormat(segment.control2.real*scale, segment.control2.imag*scale)
#             p3 = aplusbiFormat(segment.end.real*scale+x_off, segment.end.imag*scale+y_off)
#             # Convert the bezier points into a parametric equation in latex format
#             equations.append("\\\\left((1-t)^3*"+ str(p0.real)+ "+3*t*(1-t)^2*"+ str(p1.real)+ "+3*t^2*(1-t)*"+ str(p2.real)+ "+t^3*"+ str(p3.real)+ ", (1-t)^3*"+ str(p0.imag)+ "+3*t*(1-t)^2*"+ str(p1.imag)+ "+3*t^2*(1-t)*"+ str(p2.imag)+ "+t^3*"+ str(p3.imag)+ ")\\\\right)")
#             # Convert the bezier points into a parametric equation in lambda format
#             regularEquations.append(lambda t: (1 - t) ** 3 * p0+ 3 * t * (1 - t) ** 2 * p1+ 3 * t ** 2 * (1 - t) * p2+ t ** 3 * p3)

#         elif isinstance(segment, svgpathtools.path.QuadraticBezier):
#             # Quadratic Bezier segment
#             p0 = aplusbiFormat(segment.start.real*scale+x_off, segment.start.imag*scale+y_off)
#             p1 = aplusbiFormat(segment.control.real*scale, segment.control.imag*scale)
#             p2 = aplusbiFormat(segment.end.real*scale+x_off, segment.end.imag*scale+y_off)
#             # Convert the bezier points into a parametric equation in latex format
#             equations.append("\\\\left((1-t)^2*"+ str(p0.real)+ "+2*t*(1-t)*"+ str(p1.real)+ "+t^2*"+ str(p2.real)+ ", (1-t)^2*"+ str(p0.imag)+ "+2*t*(1-t)*"+ str(p1.imag)+ "+t^2*"+ str(p2.imag)+ ")\\\\right))")
#             # Convert the bezier points into a parametric equation in lambda format
#             regularEquations.append(lambda t: (1 - t) ** 2 * p0 + 2 * t * (1 - t) * p1 + t ** 2 * p2)
#         elif isinstance(segment, svgpathtools.path.Arc):
#             # Elliptical arc segment
#             p0 = aplusbiFormat(segment.start.real*scale+x_off, segment.start.imag*scale+y_off)
#             p1 = aplusbiFormat(segment.end.real*scale+x_off, segment.end.imag*scale+y_off)
#             r = aplusbiFormat(segment.radius.real*scale+x_off, segment.radius.imag*scale+y_off)
#             # Convert the bezier points into a parametric equation in latex format
#             equations.append("\\\\left("+ str(p0.real)+ "+"+ str(r.real)+ "*\\cos(t), "+ str(p0.imag)+ "+"+ str(r.imag)+ "*\\sin(t)\\\\right)")
#             # Convert the bezier points into a parametric equation in lambda format
#             regularEquations.append(lambda t: p0 + r * np.exp(1j * t))
#         else: print("Unknown segment type: " + str(type(segment)))
#     return equations
# equations = run(data,x_off=-3,y_off=5,scale=0.001)


# import svgutils
# svg = svgutils.transform.fromfile(IMAGE_NAME)
# originalSVG = svgutils.compose.SVG(IMAGE_NAME)
# # originalSVG.rotate(90)
# # originalSVG.move(svg.height, 0.01)
# originalSVG.moveto(scale= 0.01)
# figure = svgutils.compose.Figure(svg.height, svg.width, originalSVG)
# figure.save('new'+IMAGE_NAME)

# import svgutils.transform as sg
# fig = sg.fromfile(IMAGE_NAME)
# fig.set_size(('200', '200'))
# fig.save('new'+IMAGE_NAME)
# with open(r"/content/"+'new'+IMAGE_NAME, "r") as file:

# # Define the Desmos API script
# desmos = """
# <meta name="viewport" content="width=device-width, initial-scale=1">
# <script src="https://www.desmos.com/api/v1.8/calculator.js?apiKey=dcb31709b452b1cf9dc26972add0fda6"></script>
# <div id="calculator" style="width: 100%; height: 100%;"></div>
# <script>
#  var elt = document.getElementById('calculator');
#  var calculator = Desmos.GraphingCalculator(elt);
# """

# # Add the bounds to the Desmos API script
# desmos += ("calculator.setMathBounds({ left: "+ str(-31)+ ", right: "+ str(17)+ ", bottom: "+ str(-11)+ ", top: "+ str(37)+ " });\n")
# # Add each equation to the Desmos API script
# for i in range(len(equations)):
#     desmos += ("calculator.setExpression({ id: 'a-slider"+ str(i)+ "', latex: '"+ equations[i]+ "', color: Desmos.Colors.BLACK });\n")
# desmos += "</script>"

# # Save and open Desmos file
# with open("spiderman.html", "w") as f:
#     f.write(desmos)
# webbrowser.open("spiderman.html", new=2)

# # print all the eauations
# for i in range(len(equations)):
#     print(equations[i].replace("\\\\", "\\"))



In [4]:

"""
1) Usage:
 - Only SVG file types are supported
 - Use https://freesvg.org/ for free SVG files
 - Or convert PNG images to SVG using https://convertio.co/png-svg/
"""

# 2) drag and drop your svg file into the 'files' drawer on the left before running this cell
# 3) rename to your image name
IMAGE_NAME = "image.svg"

with open(r"/content/"+IMAGE_NAME, "r") as file:
    data = str(file.read()).replace('fill="#000000" opacity="1.000000" stroke="none"', "")

equations = run(data)

# save the equations to a file
with open("equations.txt", "w") as f:
    for i in range(len(equations)):
        f.write(equations[i].replace("\\\\", "\\") + "\n")

# 4) desmos equations are in the 'equations.txt' file. can double click it than copy and paste it into Desmos

